#### Sample log

In [1]:
%%bash
#strace -f -tt -T -r -o logs/mmarrays.st python src/mmarrays.py 100000
strace -f -tt -T -r -o logs/multi_threaded.st python src/multi_threaded.py

In [2]:
#!cat logs/mmarrays.st
!cat logs/multi_threaded.st

61115 15:15:02.816225 (+     0.000000) execve("/opt/conda/envs/hdf5-tutorial/bin/python", ["python", "src/multi_threaded.py"], 0x7ffeb883df98 /* 59 vars */) = 0 <0.000409>
61115 15:15:02.817325 (+     0.000931) brk(NULL) = 0x55c4ea824000 <0.000055>
61115 15:15:02.817614 (+     0.000307) readlink("/proc/self/exe", "/opt/conda/envs/hdf5-tutorial/bi"..., 4096) = 44 <0.000125>
61115 15:15:02.818077 (+     0.000447) access("/etc/ld.so.preload", R_OK) = -1 ENOENT (No such file or directory) <0.000080>
61115 15:15:02.818371 (+     0.000287) openat(AT_FDCWD, "/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/x86_64/libpthread.so.0", O_RDONLY|O_CLOEXEC) = -1 ENOENT (No such file or directory) <0.000061>
61115 15:15:02.818674 (+     0.000308) stat("/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/x86_64", 0x7fff4ba9d320) = -1 ENOENT (No such file or directory) <0.000053>
61115 15:15:02.819034 (+     0.000355) openat(AT_FDCWD, "/opt/conda/envs/hdf5-tutorial/bin/../lib/tls/haswell/libpthrea

#### Development

In [3]:
inp_str = "39209 12:49:57.788173 (+     0.000075) openat(AT_FDCWD, \"/opt/conda/envs/hdf5-tutorial/bin/../lib/libpthread.so.0\", O_RDONLY|O_CLOEXEC) = -1 ENOENT (No such file or directory) <0.000033"
attrs_ = inp_str.split()
attrs_

['39209',
 '12:49:57.788173',
 '(+',
 '0.000075)',
 'openat(AT_FDCWD,',
 '"/opt/conda/envs/hdf5-tutorial/bin/../lib/libpthread.so.0",',
 'O_RDONLY|O_CLOEXEC)',
 '=',
 '-1',
 'ENOENT',
 '(No',
 'such',
 'file',
 'or',
 'directory)',
 '<0.000033']

In [4]:
#LOG_FILE =  "logs/mmarrays.st"
LOG_FILE =  "logs/multi_threaded.st"

In [5]:
st_log_ = LOG_FILE

with open(st_log_,'r') as f:
    for line in f:
        if '<unfinished' in line:
            print(line.strip().split())
        elif 'resumed>)' in line:
            print(line.strip().split())

['77423', '16:56:40.424017', '(+', '0.000307)', 'mmap(NULL,', '8392704,', 'PROT_NONE,', 'MAP_PRIVATE|MAP_ANONYMOUS|MAP_STACK,', '-1,', '0', '<unfinished', '...>']
['77424', '16:56:40.424041', '(+', '0.000018)', 'set_robust_list(0x7f0bf06989e0,', '24', '<unfinished', '...>']
['77423', '16:56:40.424057', '(+', '0.000015)', '<...', 'mmap', 'resumed>)', '=', '0x7f0bef697000', '<0.000026>']
['77424', '16:56:40.424190', '(+', '0.000082)', '<...', 'set_robust_list', 'resumed>)', '=', '0', '<0.000143>']
['77423', '16:56:40.424209', '(+', '0.000016)', 'clone(child_stack=0x7f0befe96fb0,', 'flags=CLONE_VM|CLONE_FS|CLONE_FILES|CLONE_SIGHAND|CLONE_THREAD|CLONE_SYSVSEM|CLONE_SETTLS|CLONE_PARENT_SETTID|CLONE_CHILD_CLEARTID', '<unfinished', '...>']
['77425', '16:56:40.424493', '(+', '0.000032)', 'set_robust_list(0x7f0befe979e0,', '24', '<unfinished', '...>']
['77423', '16:56:40.424511', '(+', '0.000018)', 'mmap(NULL,', '8392704,', 'PROT_NONE,', 'MAP_PRIVATE|MAP_ANONYMOUS|MAP_STACK,', '-1,', '0', '<unf

In [4]:
reg = {}

def get_tid_time(attrs):
    tid = int(attrs[0])
    time = attrs[1]
    
    ret = [tid,time]
    return ret

def get_duration(attrs):
    dur = attrs[-1][1:-1]
    try:
        dur=float(dur)
    except ValueError:
        dur=-2.0
    return dur

def parse_call_attrs(call,unfinished,resumed,error=False):
    return [3,3,3]

def process_line(line):
    attrs = line.strip().split()
    tid,time = get_tid_time(attrs)
    if '<unfinished' in attrs:
        call = attrs[4].split('(')[0]
        call_attrs = parse_call_attrs(call,True,False)
        reg['{}_{}'.format(tid,call)] = [time,]+call_attrs 
        ret = None
    elif 'resumed>)' in attrs:
        call = attrs[5]
        rid = '{}_{}'.format(tid,call)
        if rid in reg.keys():
            tmp = reg[rid]
            time = tmp[0]
            call_attrs = tmp[1:] + parse_call_attrs(call,False,True)
            reg.pop(rid)
            dur = get_duration(attrs)
            ret = [tid,call,time,dur]+call_attrs
        else:
            call_attrs = parse_call_attrs(call,False,False,error=True)
            ret = [tid,call,time,-1.0]+call_attrs
    else:
        call = attrs[4].split('(')[0]
        call_attrs = parse_call_attrs(call,False,False)
        dur = get_duration(attrs)
        ret = [tid,call,time,dur]+call_attrs
    return ret
    
with open(st_log_,'r') as f:
    for line in f:
        ret = process_line(line)
            
        print(ret)

[77423, 'execve', '16:56:40.307923', 0.000235, 3, 3, 3]
[77423, 'brk', '16:56:40.308447', 5.9e-05, 3, 3, 3]
[77423, 'readlink', '16:56:40.308921', 0.000123, 3, 3, 3]
[77423, 'access', '16:56:40.309237', 9.7e-05, 3, 3, 3]
[77423, 'openat', '16:56:40.309488', 0.000104, 3, 3, 3]
[77423, 'stat', '16:56:40.309798', 0.000106, 3, 3, 3]
[77423, 'openat', '16:56:40.309997', 5.7e-05, 3, 3, 3]
[77423, 'stat', '16:56:40.310126', 5.6e-05, 3, 3, 3]
[77423, 'openat', '16:56:40.310289', 4.9e-05, 3, 3, 3]
[77423, 'stat', '16:56:40.310477', 0.000144, 3, 3, 3]
[77423, 'openat', '16:56:40.310765', 0.000152, 3, 3, 3]
[77423, 'stat', '16:56:40.311028', 8.1e-05, 3, 3, 3]
[77423, 'openat', '16:56:40.311294', 7.8e-05, 3, 3, 3]
[77423, 'stat', '16:56:40.311464', 0.000127, 3, 3, 3]
[77423, 'openat', '16:56:40.311958', 0.000154, 3, 3, 3]
[77423, 'stat', '16:56:40.312420', 7.1e-05, 3, 3, 3]
[77423, 'openat', '16:56:40.312671', 0.000192, 3, 3, 3]
[77423, 'stat', '16:56:40.313148', 0.000172, 3, 3, 3]
[77423, 'openat

#### Class: LineReader

In [1]:
#%%writefile ../strace_miner/preprocessor/strace_line_reader.py
import re

class StraceLineReader:
    def __init__(self):
        self.line = ""
        self.reg = {}

    def set_line(self,line):
        self.line = line

    def match_pattern(self,pattern):
        match = re.search(pattern, self.line)
        return match.group(1)

    def get_tid_time(self,attrs):
        tid = int(attrs[0])
        time = attrs[1]
        return [tid,time]

    def get_duration(self, attrs):
        dur = attrs[-1][1:-1]
        try:
            dur=float(dur)
        except ValueError:
            dur=-2.0
        return dur

    def parse_call_attrs(call,unfinished,resumed,error=False):
        return []

    def _get_combined_list(self,a,b):
        return [x if x is not None else y for x, y in zip(a, b) if x is not None or y is not None]

    def process_line(self,line):
        self.set_line(line)
        attrs = line.strip().split()
        tid,time = self.get_tid_time(attrs)
        if '<unfinished' in attrs:
            call = attrs[4].split('(')[0]
            call_attrs = self.parse_call_attrs(call,True,False)
            self.reg['{}_{}'.format(tid,call)] = [time,]+call_attrs 
            ret = None
        elif 'resumed>)' in attrs:
            call = attrs[5]
            rid = '{}_{}'.format(tid,call)
            if rid in self.reg.keys():
                tmp = self.reg[rid]
                time = tmp[0]
                tmp2 =  self.parse_call_attrs(call,False,True)
                call_attrs = self._get_combined_list(tmp[1:],tmp2)
                #call_attrs = tmp[1:] + self.parse_call_attrs(call,False,True)
                self.reg.pop(rid)
                dur = self.get_duration(attrs)
                ret = [tid,call,time,dur]+call_attrs
            else:
                call_attrs = self.parse_call_attrs(call,False,False,error=True)
                ret = [tid,call,time,-1.0]+call_attrs
        else:
            call = attrs[4].split('(')[0]
            call_attrs = self.parse_call_attrs(call,False,False)
            dur = self.get_duration(attrs)
            ret = [tid,call,time,dur]+call_attrs
        return ret  

#### Test class

In [2]:
lr = StraceLineReader()

In [6]:
with open(st_log_,'r') as f:
    for line in f:
        ret = lr.process_line(line)
        print(ret)

[77423, 'execve', '16:56:40.307923', 0.000235]
[77423, 'brk', '16:56:40.308447', 5.9e-05]
[77423, 'readlink', '16:56:40.308921', 0.000123]
[77423, 'access', '16:56:40.309237', 9.7e-05]
[77423, 'openat', '16:56:40.309488', 0.000104]
[77423, 'stat', '16:56:40.309798', 0.000106]
[77423, 'openat', '16:56:40.309997', 5.7e-05]
[77423, 'stat', '16:56:40.310126', 5.6e-05]
[77423, 'openat', '16:56:40.310289', 4.9e-05]
[77423, 'stat', '16:56:40.310477', 0.000144]
[77423, 'openat', '16:56:40.310765', 0.000152]
[77423, 'stat', '16:56:40.311028', 8.1e-05]
[77423, 'openat', '16:56:40.311294', 7.8e-05]
[77423, 'stat', '16:56:40.311464', 0.000127]
[77423, 'openat', '16:56:40.311958', 0.000154]
[77423, 'stat', '16:56:40.312420', 7.1e-05]
[77423, 'openat', '16:56:40.312671', 0.000192]
[77423, 'stat', '16:56:40.313148', 0.000172]
[77423, 'openat', '16:56:40.313515', 7.6e-05]
[77423, 'stat', '16:56:40.313808', 9.9e-05]
[77423, 'openat', '16:56:40.314074', 6.8e-05]
[77423, 'fstat', '16:56:40.314321', 8.5e-